## Remember to clear the pycache folder if the model changes

In [1]:
from LDR3_MODEL import LDR3_MODEL
import collections
import numpy as np
import time
import csv

bigDq = collections.deque(maxlen=20)  #for the prediction
smlDq = collections.deque(maxlen=18)  #for the N row moving average
sleepInterval = .005
rf_model =LDR3_MODEL() 

test=[]


def getCSVData(fname):
    List_data=[]
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            List_data.append(row)
            line_count+=1  
    return List_data



cursor = 0
def getDataFromLDR():
    global cursor
    channelArray = np.array(test[cursor][1:4]).astype('int64')
    cursor+=1
    return channelArray


directionClasses = { 0: 'up',
                     1: 'upright',
                     2: 'right',
                     3: 'downright',
                     4: 'down',
                     5: 'downleft',
                     6: 'left',
                     7: 'upleft'}



C:\Users\DannyScott\iPython Notebooks\COSC 522\FinalProject\COSC522_FinalProject\src


In [2]:
#this is UP
Cx = (10.98897041,)
Cy = (8.06188478,)
Cz = (6.78354568,)

pred_data = np.array(list([Cx[0], Cy[0], Cz[0]])).reshape(1, -1)

d = rf_model.model_predict(pred_data)
directionClasses.get(d)

'upright'

In [3]:
Cx = (8.720174194973296,)
Cy = (8.704040594174783,)
Cz = (8.655847969921284,)

pred_data = np.array(list([Cx[0], Cy[0], Cz[0]])).reshape(1, -1)
d = rf_model.model_predict(pred_data)
directionClasses.get(d)

'upleft'

In [4]:
def writeoutdata(data):
    with open("changes.txt", 'a') as file1:
        if type(data) == list:
            file1.write(','.join(str(item) for item in data)) 
            file1.write('\n')
        else:
            file1.write(data + '\n')

    

In [5]:
import pandas as pd
from scipy import ndimage

def predictDirection(aa):
    dfObj = pd.DataFrame(aa) 
    Cx = ndimage.center_of_mass(dfObj[0].values)
    Cy = ndimage.center_of_mass(dfObj[1].values)
    Cz = ndimage.center_of_mass(dfObj[2].values)
    # writeoutdata('center')       #DEBUG CODE
    # pred_data = list([Cx[0], Cy[0], Cz[0]])
    pred_data = np.array(list([Cx[0], Cy[0], Cz[0]])).reshape(1, -1)
    # writeoutdata(pred_data)     #DEBUG CODE
    d = rf_model.model_predict(pred_data)
    direction = directionClasses.get(d)
    return direction


In [6]:
#load test data
# test = getCSVData('..//data//raw_data_do_not_edit//test_data.txt')
test = getCSVData('..//data//raw_data_do_not_edit//output_3_D4_b.txt')

print(type(test))

<class 'list'>


In [7]:
print(getDataFromLDR())
cursor=0

[10048 10368  9984]


In [8]:
roll_past = 0
prev_average = None
flag_data_predictor=False

while True:
    data = getDataFromLDR()
    smlDq.append(data)
    time.sleep(sleepInterval)
    new_average = np.average(smlDq, axis=0)
    if prev_average is None:
        prev_average = new_average
    percent_change = np.subtract(new_average, prev_average)/prev_average
    list_percent_change = list(percent_change)
    # writeoutdata(list_percent_change)     #DEBUG CODE
    prev_average=new_average
    bigDq.append(list_percent_change)  #stack the percent change
    
    if (any(np.array(list_percent_change)>=0.01)):
        if (flag_data_predictor==False):
            writeoutdata('percent change')
        flag_data_predictor = True
    
    if (flag_data_predictor):    
        if(roll_past >= 18):
            p = (predictDirection(bigDq))
            # writeoutdata(p)      #DEBUG CODE
            print(p)
            roll_past = 0
            flag_data_predictor=False
        else:
            # print(roll_past)
            roll_past+=1
    else:
        continue
    


down
down
down
down
down
down
down
down
down
down
down
down
down
down
down


KeyboardInterrupt: 